# Introduction

### What is BERT? (Bidirectional Encoder Representations from Transformers)

***BERT is a deep learning model developed by Google in 2018.***  
  
***It is based on the Transformer architecture.***  
  
***It reads entire sentences at once, from both left and right — this is called bidirectional.***

For more information, the original paper can be found [here](https://arxiv.org/abs/1810.04805).

[HuggingFace documentation](https://huggingface.co/transformers/model_doc/bert.html)

[Bert documentation](https://characters.fandom.com/wiki/Bert_(Sesame_Street) ;)


# Exploratory Data Analysis and Preprocessing

We will use the SMILE Twitter dataset.

_Wang, Bo; Tsakalidis, Adam; Liakata, Maria; Zubiaga, Arkaitz; Procter, Rob; Jensen, Eric (2016): SMILE Twitter Emotion dataset. figshare. Dataset. https://doi.org/10.6084/m9.figshare.3187909.v2_

In [ ]:
import torch # Imports the PyTorch library which is a popular deep learning framework used for building and training neural networks.
import pandas as pd
from tqdm.notebook import tqdm
pd.set_option('display.max_rows', None)

In [ ]:
df = pd.read_csv('C://210878//Data Science Lab//smileannotationsfinal.csv', names=['id', 'text', 'category'])
df.set_index('id', inplace=True)

In [ ]:
df.head()

,text,category
id,,
611857364396965889,@aandraous @britishmuseum @AndrewsAntonio Merc...,nocode
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
614877582664835073,@Sofabsports thank you for following me back. ...,happy
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy


In [ ]:
df.category.value_counts()

category
nocode               1572
happy                1137
not-relevant          214
angry                  57
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|disgust             2
sad|angry               2
sad|disgust|angry       1
Name: count, dtype: int64

In [ ]:
df = df[~df.category.str.contains('\|')]

In [ ]:
df = df[df.category != 'nocode']

In [ ]:
df.category.value_counts()

category
happy           1137
not-relevant     214
angry             57
surprise          35
sad               32
disgust            6
Name: count, dtype: int64

In [ ]:
possible_labels = df.category.unique()

In [ ]:
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

In [ ]:
df['label'] = df.category.replace(label_dict)

C:\Users\Sagheer\AppData\Local\Temp\ipykernel_560\2696437164.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['label'] = df.category.replace(label_dict)


In [ ]:
df.head()

,text,category,label
id,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0


# Training/Validation Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(df.index.values,
                                                  df.label.values,
                                                  test_size=0.15,
                                                  random_state=17,
                                                  stratify=df.label.values)

In [ ]:
df['data_type'] = ['not_set']*df.shape[0]

In [ ]:
df

,text,category,label,data_type
id,,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0,not_set
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0,not_set
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0,not_set
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0,not_set
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0,not_set
...,...,...,...,...
611258135270060033,@_TheWhitechapel @Campaignforwool @SlowTextile...,not-relevant,1,not_set
612214539468279808,“@britishmuseum: Thanks for ranking us #1 in @...,happy,0,not_set
613678555935973376,MT @AliHaggett: Looking forward to our public ...,happy,0,not_set


In [ ]:
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [ ]:
df.groupby(['category', 'label', 'data_type']).count()

text
category     label data_type      
angry        2     train        48
                   val           9
disgust      3     train         5
                   val           1
happy        0     train       966
                   val         171
not-relevant 1     train       182
                   val          32
sad          4     train        27
                   val           5
surprise     5     train        30
                   val           5

# Loading Tokenizer and Encoding our Data

In [ ]:
from transformers import BartTokenizer
from torch.utils.data import TensorDataset


In [ ]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    padding='max_length',  # 'pad_to_max_length' is deprecated in newer versions
    max_length=256,
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    padding='max_length',  # 'pad_to_max_length' is deprecated in newer versions
    max_length=256,
    return_tensors='pt'
)

# The rest remains similar
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\Sagheer\anaconda3\envs\NLP\lib\site-packages\transformers\tokenization_utils_base.py:2700: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [ ]:
len(dataset_train)

1258

In [ ]:
len(dataset_val)

223

# Setting up BART Pretrained Model

In [ ]:
from transformers import BartForSequenceClassification

In [ ]:
model = BartForSequenceClassification.from_pretrained("facebook/bart-base",
                                                     num_labels=len(label_dict),
                                                     output_attentions=False,
                                                     output_hidden_states=False)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Creating Data Loaders

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [ ]:
batch_size = 32

dataloader_train = DataLoader(dataset_train,
                              sampler=RandomSampler(dataset_train),
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val,
                                   sampler=SequentialSampler(dataset_val),
                                   batch_size=batch_size)

# Setting Up Optimiser and Scheduler

In [ ]:
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr=1e-5,
                  eps=1e-8)

In [ ]:
epochs = 3

scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

# Defining our Performance Metrics

In [ ]:
# Import necessary libraries
import numpy as np  # NumPy for numerical operations
from sklearn.metrics import f1_score  # Import the F1 score metric from scikit-learn

In [ ]:
def f1_score_func(preds, labels):
    """
    Calculate the weighted F1 score for multi-class classification.

    Parameters:
    -----------
    preds : numpy.ndarray
        Prediction probabilities array with shape (n_samples, n_classes)
        where each row contains the probability scores for all classes

    labels : numpy.ndarray
        Ground truth labels with shape (n_samples,)

    Returns:
    --------
    float
        Weighted F1 score
    """
    # Convert prediction probabilities to class indices by taking argmax along class dimension
    # and flatten the result to a 1D array
    preds_flat = np.argmax(preds, axis=1).flatten()

    # Flatten the ground truth labels to ensure they're in the same format as predictions
    labels_flat = labels.flatten()

    # Calculate and return the weighted F1 score
    # 'weighted' averages the F1 scores for each class, weighted by the number of true instances of each class
    return f1_score(labels_flat, preds_flat, average='weighted')

In [ ]:
def accuracy_per_class(preds, labels):
    """
    Calculate and print the accuracy for each class separately.

    Parameters:
    -----------
    preds : numpy.ndarray
        Prediction probabilities array with shape (n_samples, n_classes)
        where each row contains the probability scores for all classes

    labels : numpy.ndarray
        Ground truth labels with shape (n_samples,)

    Note:
    -----
    This function assumes a global 'label_dict' exists that maps
    class indices to human-readable class names.
    """
    # Create an inverse mapping from numeric label indices to human-readable class names
    # This reverses the label_dict mapping for displaying class names in the output
    label_dict_inverse = {v: k for k, v in label_dict.items()}

    # Convert prediction probabilities to class indices by taking argmax along class dimension
    # and flatten the result to a 1D array
    preds_flat = np.argmax(preds, axis=1).flatten()

    # Flatten the ground truth labels to ensure they're in the same format as predictions
    labels_flat = labels.flatten()

    # Iterate through each unique class label in the ground truth
    for label in np.unique(labels_flat):
        # Extract predictions for samples that actually belong to the current class
        y_preds = preds_flat[labels_flat==label]

        # Extract ground truth labels for samples that belong to the current class
        # (this will be all the same value, equal to 'label')
        y_true = labels_flat[labels_flat==label]

        # Print the human-readable class name
        print(f'Class: {label_dict_inverse[label]}')

        # Print the accuracy for this class:
        # - Count correct predictions (where predicted label equals the true label)
        # - Divide by total number of samples in this class
        # - Format as a fraction (correct/total)
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

# Creating our Training Loop

In [ ]:
import random  # Import Python's random module for basic random number generation

# Set a specific seed value for reproducibility across runs
seed_val = 17

# Set seed for Python's built-in random module
# This affects functions like random.random(), random.choice(), etc.
random.seed(seed_val)

# Set seed for NumPy's random number generator
# This affects functions like np.random.rand(), np.random.shuffle(), etc.
np.random.seed(seed_val)

# Set seed for PyTorch's CPU random number generator
# This affects random operations in PyTorch when running on CPU
torch.manual_seed(seed_val)

# Set seed for PyTorch's GPU random number generator on all available GPUs
# This ensures reproducibility when running on GPU
# It's safe to call this even if no GPU is available
torch.cuda.manual_seed_all(seed_val)

In [ ]:
# Determine which device (GPU or CPU) to use for training
# If a CUDA-compatible GPU is available, use it; otherwise fall back to CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move the model to the selected device (GPU or CPU)
# This transfers all model parameters (weights and biases) to the appropriate device
# For large models, this can significantly speed up training when using GPU
model.to(device)

# Print the selected device to confirm where computation will occur
# Will output either "cuda" (for GPU) or "cpu"
print(device)

cpu


In [ ]:
def evaluate(dataloader_val):
    """
    Evaluate the model on validation/test data.

    Parameters:
    -----------
    dataloader_val : torch.utils.data.DataLoader
        DataLoader containing validation/test batches

    Returns:
    --------
    tuple
        (average_loss, predictions, true_labels)
        where predictions and true_labels are numpy arrays
    """
    # Set the model to evaluation mode
    # This disables dropout, batch normalization updates, etc.
    model.eval()

    # Initialize total validation loss
    loss_val_total = 0

    # Lists to store predictions and true labels for all batches
    predictions, true_vals = [], []

    # Iterate over all batches in the validation dataloader
    for batch in dataloader_val:

        # Move each tensor in the batch to the specified device (GPU/CPU)
        # This unpacks the batch tuple and moves each component to the device
        batch = tuple(b.to(device) for b in batch)

        # Create inputs dictionary expected by the model
        # Typical structure for transformer models (e.g., BERT, RoBERTa)
        inputs = {'input_ids':      batch[0],  # Token IDs
                  'attention_mask': batch[1],  # Attention mask for padding
                  'labels':         batch[2],  # Ground truth labels
                 }

        # Disable gradient calculation for evaluation to save memory and computation
        with torch.no_grad():
            # Forward pass through the model
            # The ** operator unpacks the dictionary as keyword arguments
            outputs = model(**inputs)

        # Extract loss and logits from model outputs
        # Structure depends on model configuration, but typically:
        # outputs[0] = loss, outputs[1] = logits
        loss = outputs[0]
        logits = outputs[1]

        # Accumulate the validation loss
        loss_val_total += loss.item()

        # Process logits:
        # 1. Detach from computation graph
        # 2. Move to CPU
        # 3. Convert to NumPy array for further processing
        logits = logits.detach().cpu().numpy()

        # Similarly convert labels to NumPy array
        label_ids = inputs['labels'].cpu().numpy()

        # Store batch predictions and true labels
        predictions.append(logits)
        true_vals.append(label_ids)

    # Calculate average validation loss
    loss_val_avg = loss_val_total/len(dataloader_val)

    # Concatenate all batch predictions into a single array
    # axis=0 concatenates along the batch dimension
    predictions = np.concatenate(predictions, axis=0)

    # Concatenate all batch true labels into a single array
    true_vals = np.concatenate(true_vals, axis=0)

    # Return average loss, predictions, and true labels
    return loss_val_avg, predictions, true_vals

In [ ]:
# Iterate through training epochs with progress tracking via tqdm
for epoch in tqdm(range(1, epochs+1)):

    # Set model to training mode
    # This enables dropout, batch normalization updates, etc.
    model.train()

    # Initialize total training loss for this epoch
    loss_train_total = 0

    # Create a progress bar for batches within this epoch
    # - desc: Shows which epoch we're on
    # - leave=False: Removes the progress bar when the epoch finishes
    # - disable=False: Ensures the progress bar is displayed
    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)

    # Iterate through each batch in the training dataloader
    for batch in progress_bar:

        # Reset gradients from previous batch
        # This prevents gradient accumulation between batches
        model.zero_grad()

        # Move each tensor in the batch to the specified device (GPU/CPU)
        batch = tuple(b.to(device) for b in batch)

        # Create inputs dictionary expected by the model
        # Typical structure for transformer models (e.g., BERT, RoBERTa)
        inputs = {'input_ids':      batch[0],  # Token IDs
                  'attention_mask': batch[1],  # Attention mask for padding
                  'labels':         batch[2],  # Ground truth labels
                 }

        # Forward pass through the model
        # The ** operator unpacks the dictionary as keyword arguments
        outputs = model(**inputs)

        # Extract loss from model outputs (typically the first element)
        loss = outputs[0]

        # Accumulate the training loss for reporting
        loss_train_total += loss.item()

        # Backward pass: compute gradients of the loss with respect to model parameters
        loss.backward()

        # Apply gradient clipping to prevent exploding gradients
        # Max norm of 1.0 ensures gradients don't get too large
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update model parameters based on computed gradients
        optimizer.step()

        # Update learning rate scheduler
        # This could be a linear scheduler, cosine scheduler, etc.
        scheduler.step()

        # Update progress bar with the current batch loss
        # Normalizing by batch size to make it comparable across different batch sizes
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})


    # Save model checkpoint at the end of each epoch
    # This allows for recovery in case of interruption and selection of best model later
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')

    # Print epoch number to the console
    tqdm.write(f'\nEpoch {epoch}')

    # Calculate and display average training loss for this epoch
    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')

    # Evaluate the model on validation data
    val_loss, predictions, true_vals = evaluate(dataloader_validation)

    # Calculate F1 score on validation data
    # This is a key metric for classification tasks, especially with imbalanced classes
    val_f1 = f1_score_func(predictions, true_vals)

    # Print validation metrics
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/40 [00:00<?, ?it/s]


KeyboardInterrupt



In [ ]:
# Initialize a BART model for sequence classification
# - This creates a model with a BART encoder-decoder and a classification head on top
model = BartForSequenceClassification.from_pretrained(
    "facebook/bart-base",  # Load the pre-trained BART base model
                          # Unlike BERT, BART preserves case by default

    num_labels=len(label_dict),  # Set the number of output classes based on our label dictionary
                                # This configures the classification head appropriately

    output_attentions=False,    # Don't return attention weights in the output
                               # Setting to False saves memory and computation

    output_hidden_states=False  # Don't return all hidden-states in the output
                               # Setting to False saves memory and computation
)

# Move the model to the appropriate device (GPU if available, otherwise CPU)
# This transfers all model parameters to the device's memory
model.to(device)

In [ ]:
# Evaluate the model on the validation dataset
# - The evaluate function returns (val_loss, predictions, true_vals)
# - We use underscore (_) to ignore the validation loss since we don't need it here
_, predictions, true_vals = evaluate(dataloader_validation)

In [ ]:
# Calculate and print the accuracy for each individual class
# This gives a detailed breakdown of model performance across different classes
# Particularly important for imbalanced datasets where overall accuracy might be misleading
accuracy_per_class(predictions, true_vals)